In [ ]:
#| hide
from nbdev_proteomics.core import *
from nbdev_proteomics.dim_red import *

# nbdev-proteomics

> process proteomic quantification

A collection of utilities to process proteomics data

## Install

```sh
pip install nbdev_proteomics
```

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2